In [1]:
#import
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import numpy as np
import pymysql
import operator
import os
from scipy import stats
from statsmodels.regression.linear_model import OLS
from datetime import *
import scipy.io as sio
import h5py
import copy
import shutil
import rarfile
import scipy.stats as scs
from general_function import *
from matplotlib_venn import venn3
from matplotlib_venn import venn2
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.font_manager import FontProperties
plt.rcParams['font.sans-serif'] = ['SimHei']  #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  #用来正常显示负号
sns.set(style="whitegrid")
myfont = FontProperties(fname=r'C:\Windows\Fonts\simhei.ttf', size=10)
sns.set(
    font=myfont.get_name(),
    style="whitegrid",
    palette="muted",
    color_codes=True)

In [2]:
#data
os.chdir(r'C:/Users/bj/Desktop/刘泽丹/Code/')
STcode = pd.read_csv('STcode_by_date_for_generator.csv',index_col=0).iloc[-1].dropna().tolist()
code_to_name = pd.read_csv('code_to_name.csv',encoding='gbk',index_col=0)['SEC_NAME']
code_to_name.index = code_to_name.index.str[:6].astype(int)
account_code_to_name = pd.read_excel('account_info.xlsx').set_index('产品标识')['账户备注']
account_code_to_name.index = [i.split(',')[0] if type(i)==str else np.nan for i in account_code_to_name.index]
account_name_to_t0code = pd.read_excel('t0编号.xlsx',index_col=0)['T0代码']

In [3]:
#today
today = datetime.today()
year = str(today.year)
month = today.month
if month < 10:
    month = '0' + str(month)
else:
    month = str(month)
day = today.day
if day < 10:
    day = '0' + str(day)
else:
    day = str(day)    
today_nodot = year + month + day
tomorrow_nodot = get_n_week_date_after(today_nodot)
yesterday_nodot = get_n_week_date_before(today_nodot)

In [4]:
today_nodot,tomorrow_nodot,yesterday_nodot

('20190822', '20190823', '20190821')

today_nodot = '20190813';tomorrow_nodot = '20190814';yesterday_nodot = '20190812'

In [5]:
back_checking_days = 12

In [6]:
#理论的卖出任务，即如*整七天前*买入标记为H的，*整六天前*买入标记为E的等，买入记录为‘实验_总任务管理表_买入_完成_DATE.xlsx’表格
theoretical_sell = pd.DataFrame()
for i in range(0, back_checking_days-1):
    print(get_n_week_date_before(today_nodot,i))
    back_updated_buy = pd.read_excel('实验_总任务管理表_买入_完成_'+get_n_week_date_before(today_nodot,i)+'.xlsx')
    date_model_list = [date_model for date_model in back_updated_buy['模型'].unique().tolist() \
                       if type(date_model) == int and date_model == int(tomorrow_nodot)]
    sell_list = []
    sell_list = preset_n_days_model(i+2, add_num=True) + date_model_list
    print(sell_list)
    sub_theoretical_sell = back_updated_buy[back_updated_buy['模型'].isin(sell_list)]
    sub_theoretical_sell['买入日期'] = get_n_week_date_before(today_nodot,i)
    theoretical_sell = pd.concat([theoretical_sell,sub_theoretical_sell])
theoretical_sell['任务资金'] = np.nan
theoretical_sell['任务股数'] = theoretical_sell['完成股数']
theoretical_sell['方向'] = '空'
theoretical_sell = theoretical_sell.loc[:,['产品标识','股票代码','任务资金','任务股数','方向','模型','买入日期']].reset_index(drop = True)

20190822
['A2', 'A3', 'A4', 'A5', 'B1', 'B3', 'B4', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'K1', 'K2', 'K3', 'K4', 'K5', 'K6', 'K7', 2, '2', 20190823]
20190821
['G1', 3, '3', 20190823]
20190820
[4, '4', 20190823]
20190819
['sxf', 5, '5', 20190823]
20190816
['E1', 'E2', 'E3', 'E4', 'F1', 6, '6', 20190823]
20190815
['H1', 'H2', 'H3', 7, '7']
20190814
[8, '8', 20190823]
20190813
['I1', 'I2', 'I3', 'I4', 'I5', 9, '9']
20190812
[10, '10']
20190809
[11, '11']
20190808
['J2', 12, '12']


In [7]:
#理论的可T任务，即如*七天以内（不含整七天）*买入标记为H的，*六天以内（不含整六天）*买入标记为E的等，买入记录为‘实验_总任务管理表_买入_完成_DATE.xlsx’表格
theoretical_t0 = pd.DataFrame()
for i in range(0, back_checking_days-2):
    print(get_n_week_date_before(today_nodot,i))
    back_updated_buy = pd.read_excel('实验_总任务管理表_买入_完成_'+get_n_week_date_before(today_nodot,i)+'.xlsx')
    date_model_list = [date_model for date_model in back_updated_buy['模型'].unique().tolist() \
                       if type(date_model) == int and date_model > int(tomorrow_nodot)]
    t0_list = preset_n_days_model_combined(range(i+3,back_checking_days+1)) + date_model_list
    t0_list += list(range(i+3,back_checking_days+1))
    print(t0_list)
    sub_theoretical_t0 = back_updated_buy[back_updated_buy['模型'].isin(t0_list)]
    sub_theoretical_t0['买入日期'] = get_n_week_date_before(today_nodot,i)
    theoretical_t0 = pd.concat([theoretical_t0,sub_theoretical_t0])
theoretical_t0['任务资金'] = np.nan
theoretical_t0['任务股数'] = theoretical_t0['完成股数']
theoretical_t0['方向'] = '空'
theoretical_t0 = theoretical_t0.loc[:,['产品标识','股票代码','任务资金','任务股数','方向','模型','买入日期']].reset_index(drop = True)

20190822
['G1', 'sxf', 'E1', 'E2', 'E3', 'E4', 'F1', 'H1', 'H2', 'H3', 'I1', 'I2', 'I3', 'I4', 'I5', 'J2', 20190902, 20190829, 20190830, 20190826, 20190827, 20190828, 20190903, 20190904, 20190905, 20190906, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
20190821
['sxf', 'E1', 'E2', 'E3', 'E4', 'F1', 'H1', 'H2', 'H3', 'I1', 'I2', 'I3', 'I4', 'I5', 'J2', 20190830, 20190829, 20190826, 20190827, 20190828, 20190902, 4, 5, 6, 7, 8, 9, 10, 11, 12]
20190820
['sxf', 'E1', 'E2', 'E3', 'E4', 'F1', 'H1', 'H2', 'H3', 'I1', 'I2', 'I3', 'I4', 'I5', 'J2', 20190829, 20190828, 5, 6, 7, 8, 9, 10, 11, 12]
20190819
['E1', 'E2', 'E3', 'E4', 'F1', 'H1', 'H2', 'H3', 'I1', 'I2', 'I3', 'I4', 'I5', 'J2', 20190828, 20190827, 6, 7, 8, 9, 10, 11, 12]
20190816
['H1', 'H2', 'H3', 'I1', 'I2', 'I3', 'I4', 'I5', 'J2', 20190827, 20190826, 20190828, 7, 8, 9, 10, 11, 12]
20190815
['I1', 'I2', 'I3', 'I4', 'I5', 'J2', 20190826, 20190827, 8, 9, 10, 11, 12]
20190814
['I1', 'I2', 'I3', 'I4', 'I5', 'J2', 9, 10, 11, 12]
20190813
['J2', 10, 11,

In [8]:
#理论的总持仓，即如*七天以内（含整七天）*买入标记为H的，*六天以内（含整六天）*买入标记为E的等，买入记录为‘实验_总任务管理表_买入_完成_DATE.xlsx’表格
theoretical_total_holding = pd.DataFrame()
for i in range(0, back_checking_days-1):
    print(get_n_week_date_before(today_nodot,i))
    back_updated_buy = pd.read_excel('实验_总任务管理表_买入_完成_'+get_n_week_date_before(today_nodot,i)+'.xlsx')
    date_model_list = [date_model for date_model in back_updated_buy['模型'].unique().tolist() \
                       if type(date_model) == int and date_model>=int(tomorrow_nodot)]
    total_holding_list = []
    total_holding_list = preset_n_days_model_combined(range(i+2,back_checking_days+1)) + date_model_list
    total_holding_list += list(range(i+2,back_checking_days+1))
    print(total_holding_list)
    sub_theoretical_total_holding = back_updated_buy[back_updated_buy['模型'].isin(total_holding_list)]
    sub_theoretical_total_holding['买入日期'] = get_n_week_date_before(today_nodot,i)
    theoretical_total_holding = pd.concat([theoretical_total_holding,sub_theoretical_total_holding])
theoretical_total_holding['任务资金'] = np.nan
theoretical_total_holding['任务股数'] = theoretical_total_holding['完成股数']
theoretical_total_holding['方向'] = '空'
theoretical_total_holding = theoretical_total_holding.loc[:,['产品标识','股票代码','任务资金','任务股数','方向','模型','买入日期']].reset_index(drop = True)

20190822
['A2', 'A3', 'A4', 'A5', 'B1', 'B3', 'B4', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'K1', 'K2', 'K3', 'K4', 'K5', 'K6', 'K7', 'G1', 'sxf', 'E1', 'E2', 'E3', 'E4', 'F1', 'H1', 'H2', 'H3', 'I1', 'I2', 'I3', 'I4', 'I5', 'J2', 20190902, 20190829, 20190830, 20190823, 20190826, 20190827, 20190828, 20190903, 20190904, 20190905, 20190906, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
20190821
['G1', 'sxf', 'E1', 'E2', 'E3', 'E4', 'F1', 'H1', 'H2', 'H3', 'I1', 'I2', 'I3', 'I4', 'I5', 'J2', 20190830, 20190829, 20190823, 20190826, 20190827, 20190828, 20190902, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
20190820
['sxf', 'E1', 'E2', 'E3', 'E4', 'F1', 'H1', 'H2', 'H3', 'I1', 'I2', 'I3', 'I4', 'I5', 'J2', 20190829, 20190828, 20190823, 4, 5, 6, 7, 8, 9, 10, 11, 12]
20190819
['sxf', 'E1', 'E2', 'E3', 'E4', 'F1', 'H1', 'H2', 'H3', 'I1', 'I2', 'I3', 'I4', 'I5', 'J2', 20190828, 20190827, 20190823, 5, 6, 7, 8, 9, 10, 11, 12]
20190816
['E1', 'E2', 'E3', 'E4', 'F1', 'H1', 'H2', 'H3', 'I1', 'I2', 'I3', 'I4', 'I5', 'J2', 2019

In [9]:
#make a copy for future usage
theoretical_sell_origin = theoretical_sell.copy()
theoretical_t0_origin = theoretical_t0.copy()
theoretical_total_holding_origin = theoretical_total_holding.copy()

In [10]:
#卖出任务中的st股
theoretical_sell[theoretical_sell['股票代码'].isin(STcode)]

,产品标识,股票代码,任务资金,任务股数,方向,模型,买入日期


In [11]:
#可T任务中的st股
theoretical_t0[theoretical_t0['股票代码'].isin(STcode)]

,产品标识,股票代码,任务资金,任务股数,方向,模型,买入日期
29,HQCC3,300431,NaN,3600,空,I4,20190822
76,HQCC3,300431,NaN,3500,空,I5,20190822
140,HQCC4,300431,NaN,2500,空,I4,20190822
187,HQCC4,300431,NaN,2500,空,I5,20190822
252,HQCC5,300431,NaN,27600,空,I4,20190822
300,HQDB2,300431,NaN,8300,空,I5,20190822
350,HQDB3,300431,NaN,2100,空,I5,20190822
410,HQDF1,300431,NaN,4100,空,I5,20190822
464,HQDT1,300431,NaN,2600,空,I4,20190822
511,HQDT1,300431,NaN,2500,空,I5,20190822


In [12]:
#将st股全部转至卖出任务中
theoretical_sell = pd.concat([theoretical_sell,theoretical_t0[theoretical_t0['股票代码'].isin(STcode)]])
theoretical_t0 = theoretical_t0[~theoretical_t0['股票代码'].isin(STcode)]

In [13]:
#验证：理论卖出+可T = 持仓，输出应为一个空Series
(theoretical_total_holding.groupby(['产品标识','股票代码']).sum()['任务股数'] - \
(theoretical_t0.groupby(['产品标识','股票代码']).sum()['任务股数']\
.loc[theoretical_total_holding.groupby(['产品标识','股票代码']).sum()['任务股数'].index].fillna(0) + \
theoretical_sell.groupby(['产品标识','股票代码']).sum()['任务股数']\
.loc[theoretical_total_holding.groupby(['产品标识','股票代码']).sum()['任务股数'].index].fillna(0)))[\
(theoretical_total_holding.groupby(['产品标识','股票代码']).sum()['任务股数'] - \
(theoretical_t0.groupby(['产品标识','股票代码']).sum()['任务股数']\
.loc[theoretical_total_holding.groupby(['产品标识','股票代码']).sum()['任务股数'].index].fillna(0) + \
theoretical_sell.groupby(['产品标识','股票代码']).sum()['任务股数']\
.loc[theoretical_total_holding.groupby(['产品标识','股票代码']).sum()['任务股数'].index].fillna(0)))!=0]

Series([], Name: 任务股数, dtype: float64)

In [14]:
#股数整合，即不同日期、以不同模型买入的相同股票进行加总
theoretical_sell_sum = theoretical_sell.groupby(['产品标识','股票代码']).sum()['任务股数'].reset_index()
theoretical_sell_sum['股票名称'] = code_to_name.loc[theoretical_sell_sum['股票代码']].tolist()
theoretical_t0_sum = theoretical_t0.groupby(['产品标识','股票代码']).sum()['任务股数'].reset_index()
theoretical_t0_sum['股票名称'] = code_to_name.loc[theoretical_t0_sum['股票代码']].tolist()
theoretical_total_holding_sum = theoretical_total_holding.groupby(['产品标识','股票代码']).sum()['任务股数'].reset_index()
theoretical_total_holding_sum['股票名称'] = code_to_name.loc[theoretical_total_holding_sum['股票代码']].tolist()

In [15]:
#筛选掉某些特殊的产品标识等，一般没有用
theoretical_total_holding_sum = theoretical_total_holding_sum[~theoretical_total_holding_sum['产品标识'].isin([])]
theoretical_sell_sum = theoretical_sell_sum[~theoretical_sell_sum['产品标识'].isin([])]
theoretical_t0_sum = theoretical_t0_sum[~theoretical_t0_sum['产品标识'].isin([])]
theoretical_total_holding_sum = theoretical_total_holding_sum[theoretical_total_holding_sum['任务股数']!=0]
theoretical_sell_sum = theoretical_sell_sum[theoretical_sell_sum['任务股数']!=0]
theoretical_t0_sum = theoretical_t0_sum[theoretical_t0_sum['任务股数']!=0]

In [16]:
#进行一下记录，一般没有用
theoretical_sell_normal_output = theoretical_sell_sum.copy()
theoretical_sell_normal_output['任务资金'] = ''
theoretical_sell_normal_output['方向'] = '空'
theoretical_sell_normal_output = theoretical_sell_normal_output.loc[:,['产品标识','股票代码','任务资金','任务股数','方向']]
theoretical_total_holding_sum.to_excel('理论持仓.xlsx',encoding='gbk',index=False)
theoretical_sell_normal_output.to_excel('理论明日卖出.xlsx',encoding='gbk',index=False)
theoretical_t0_sum.to_excel('理论明日可T.xlsx',encoding='gbk',index=False)

In [17]:
#卖出、可T、持仓变为以产品标识和股票代码为index的任务股数Series
theoretical_total_holding_sum = theoretical_total_holding_sum.set_index(['产品标识','股票代码'])['任务股数'].sort_index()
theoretical_sell_sum = theoretical_sell_sum.set_index(['产品标识','股票代码'])['任务股数'].sort_index()
theoretical_t0_sum = theoretical_t0_sum.set_index(['产品标识','股票代码'])['任务股数'].sort_index()

In [ ]:
#如第一次运行，运行第1,2,3步，不运行第4步！
#第一次运行/对持仓文件手动修改后，不运行第1步！按顺序运行第4,2,3步！

In [ ]:
#1读取昨天的持仓，加减今日的买入和卖出
os.chdir(r'C:/Users/bj/Desktop/刘泽丹/Code')
real_total_holding = pd.read_excel(yesterday_nodot+'持仓.xlsx',encoding='gbk')
real_total_holding.to_excel(today_nodot+'持仓.xlsx',encoding='gbk',index = False)
real_total_holding = pd.read_excel(today_nodot+'持仓.xlsx',encoding='gbk')
todate_buyNsell = pd.read_excel('任务管理_管理员_'+today_nodot[:4]+'-'+today_nodot[4:6]+'-'+today_nodot[6:]+'.xlsx')
todate_buyNsell['方向'] = todate_buyNsell['方向'].replace('多',1).replace('空',-1)
todate_buyNsell = todate_buyNsell.set_index(['产品标识','股票代码'])['方向'] * todate_buyNsell.set_index(['产品标识','股票代码'])['完成股数']
todate_buyNsell.name = '任务股数'
todate_buyNsell = todate_buyNsell.reset_index()
real_total_holding = pd.concat([real_total_holding,todate_buyNsell])

In [19]:
#2股数整合
real_total_holding = real_total_holding.set_index(['产品标识','股票代码'])['任务股数'].sort_index()
real_total_holding = real_total_holding.groupby(level=[0,1]).sum()
real_total_holding = real_total_holding.replace(0,np.nan).dropna()
os.chdir(r'C:/Users/bj/Desktop/刘泽丹/Code')
real_total_holding.name = '任务股数'

In [20]:
real_total_holding.sort_values()

产品标识    股票代码  
HQHF1   2442           3.0
HQHZ7   2010           3.0
HQCC5   600585         4.0
HQHZ20  300552         5.0
        300648         6.0
        300554         6.0
HQCC5   300375         8.0
HQCC4   2484          10.0
HQHF9   2422          10.0
HQHZ20  300191        10.0
        300578        10.0
HQYH25  600161        12.0
HQHZ7   300206        20.0
        601229        20.0
HQXB2   603816        20.0
HQHZ20  2035          25.0
HQHZ7   600570        27.0
HQHZ20  2530          30.0
HQCC5   300536        36.0
HQHZ20  600338        39.0
HQYH23  600155        39.0
HQGX1   2761          40.0
HQYH11  600751        40.0
HQHZ20  300029        40.0
HQYH23  300346        40.0
HQGD1   655           40.0
HQGX1   600984        40.0
HQHZ20  300033        43.0
        2869          43.0
HQYH9   2640          44.0
                    ...   
HQGX8   300325    388800.0
HQYH23  600300    390400.0
        300459    396500.0
HQGX8   601777    402500.0
        2239      406800.0
HQYH25  60074

In [21]:
print(int(real_total_holding.sum()))

176839989


In [22]:
#3
real_total_holding.reset_index().to_excel(today_nodot+'持仓.xlsx',encoding='gbk',index = False)

In [18]:
#4
real_total_holding = pd.read_excel(today_nodot+'持仓.xlsx',encoding='gbk')

In [ ]:
yesteryesterday_nodot = get_n_week_date_before(yesterday_nodot)

In [ ]:
#今天和昨天的买入卖出任务
todate_buyNsell = pd.read_excel('任务管理_管理员_'+today_nodot[:4]+'-'+today_nodot[4:6]+'-'+today_nodot[6:]+'.xlsx')
yesterdate_buyNsell = pd.read_excel('任务管理_管理员_'+yesterday_nodot[:4]+'-'+yesterday_nodot[4:6]+'-'+yesterday_nodot[6:]+'.xlsx')
yesteryesterdate_buyNsell = pd.read_excel('任务管理_管理员_'+yesteryesterday_nodot[:4]+'-'+yesteryesterday_nodot[4:6]+'-'+yesteryesterday_nodot[6:]+'.xlsx')

In [ ]:
#标记今天和昨天的卖出未完成部分
todate_unfinished = todate_buyNsell[(todate_buyNsell['方向']=='空') & (todate_buyNsell['任务进度']!=1)]\
    .set_index(['产品标识','股票代码']).loc[:,['任务股数','完成股数','产品别称','股票名称']]
yesterdate_unfinished = yesterdate_buyNsell[(yesterdate_buyNsell['方向']=='空') & (yesterdate_buyNsell['任务进度']!=1)]\
    .set_index(['产品标识','股票代码']).loc[:,['任务股数','完成股数','产品别称','股票名称']]
yesteryesterdate_unfinished = yesteryesterdate_buyNsell[(yesteryesterdate_buyNsell['方向']=='空') & (yesteryesterdate_buyNsell['任务进度']!=1)]\
    .set_index(['产品标识','股票代码']).loc[:,['任务股数','完成股数','产品别称','股票名称']]
todate_unfinished['今日未完成股数'] = todate_unfinished['任务股数'] - todate_unfinished['完成股数']
yesterdate_unfinished['昨日未完成股数'] = yesterdate_unfinished['任务股数'] - yesterdate_unfinished['完成股数']
yesteryesterdate_unfinished['前日未完成股数'] = yesteryesterdate_unfinished['任务股数'] - yesteryesterdate_unfinished['完成股数']
unfinished_compare = pd.concat([yesteryesterdate_unfinished,yesterdate_unfinished,todate_unfinished],axis=1).dropna()

In [ ]:
#如果今天和昨天的卖出未完成部分完全相同，则可能是真实持仓和理论持仓的差异造成的
unfinished_compare = unfinished_compare[(unfinished_compare['今日未完成股数']==unfinished_compare['昨日未完成股数'])&\
                                        (unfinished_compare['今日未完成股数']==unfinished_compare['前日未完成股数'])].reset_index()

In [ ]:
#输出一下，可在同花顺等软件内进行一下验证，如果是因为跌停、停牌等原因真的不能卖出的则不能算是记录错误
['0'*(6-len(str(i)))+str(i) for i in sorted(unfinished_compare['股票代码'].unique().tolist())]

In [ ]:
unfinished_compare

In [ ]:
#去掉上述特殊处理的
unfinished_compare = unfinished_compare[~unfinished_compare['产品标识'].isin([])]
unfinished_compare = unfinished_compare[~unfinished_compare['股票代码'].isin([895,])]

In [ ]:
unfinished_compare.reset_index(drop=True)

In [ ]:
#从持仓记录中去掉推测的记录错误内容
inferred_actually_sold = unfinished_compare.loc[:,['产品标识','股票代码','今日未完成股数']]
real_total_holding = pd.read_excel(today_nodot+'持仓.xlsx',encoding='gbk')
inferred_actually_sold.columns = ['产品标识','股票代码','任务股数']
inferred_actually_sold['任务股数'] = -inferred_actually_sold['任务股数']
real_total_holding = pd.concat([real_total_holding,inferred_actually_sold])
real_total_holding = real_total_holding.set_index(['产品标识','股票代码'])['任务股数'].sort_index()
real_total_holding = real_total_holding.groupby(level=[0,1]).sum()
real_total_holding = real_total_holding.replace(0,np.nan).dropna()
os.chdir(r'C:/Users/bj/Desktop/刘泽丹/Code')
real_total_holding.name = '任务股数'

In [ ]:
real_total_holding.sort_values()

In [ ]:
int(real_total_holding.sum())

In [ ]:
real_total_holding.reset_index().to_excel(today_nodot+'持仓.xlsx',encoding='gbk',index = False)

In [23]:
#理论持仓、理论卖出、理论可T和真实持仓的df
result = pd.concat([theoretical_total_holding_sum,theoretical_sell_sum,theoretical_t0_sum,real_total_holding],axis=1).fillna(0)
result.columns = ['ty_hold','ty_sell','ty_t0','real_hold']

In [24]:
result

ty_hold  ty_sell    ty_t0  real_hold
产品标识  股票代码                                        
HQCC3 1       12800.0   2400.0  10400.0    12800.0
      35          0.0      0.0      0.0      100.0
      46      20300.0   4300.0  16000.0    20300.0
      56      13500.0      0.0  13500.0    13500.0
      69       2800.0      0.0   2800.0     2800.0
      150      3600.0      0.0   3600.0     3600.0
      157     12700.0      0.0  12700.0    12700.0
      159     23200.0   6000.0  17200.0    23200.0
      401     10200.0   2400.0   7800.0    10200.0
      425     18800.0      0.0  18800.0    18800.0
      503      5600.0      0.0   5600.0     5600.0
      507      6100.0      0.0   6100.0     6100.0
      513       700.0      0.0    700.0      700.0
      516     13700.0  13700.0      0.0    13700.0
      568      2400.0    400.0   2000.0     2400.0
      576      2100.0      0.0   2100.0     2100.0
      596      1700.0    200.0   1500.0     1700.0
      598     14800.0  14800.0      0.0    14800.0
      610      6100.0      0.0   6100.0     6100.0
      612      4200.0   4200.0      0.0     4200.0
      655     14000.0   7400.0   6600.0    14000.0
      656      2500.0      0.0   2500.0     2500.0
      661       200.0      0.0    200.0      200.0
      708      3700.0      0.0   3700.0     3700.0
      716     10700.0      0.0  10700.0    10700.0
      723      3800.0      0.0   3800.0     3800.0
      729     11200.0      0.0  11200.0    11200.0
      732      8400.0      0.0   8400.0     8400.0
      786      3100.0      0.0   3100.0     3100.0
      799       600.0      0.0    600.0      600.0
...               ...      ...      ...        ...
HQZS2 600278   1500.0      0.0   1500.0     1500.0
      600300  16000.0      0.0  16000.0    16000.0
      600393  50400.0   7300.0  43100.0    50400.0
      600410   1600.0   1600.0      0.0     1600.0
      600552   3600.0      0.0   3600.0     3600.0
      600600   1800.0    200.0   1600.0     1800.0
      600604  15200.0   1900.0  13300.0    15200.0
      600677   7200.0   1200.0   6000.0     7200.0
      600682   4400.0      0.0   4400.0     4400.0
      600720   5900.0      0.0   5900.0     5900.0
      600744  26400.0      0.0  26400.0    26400.0
      600855   1800.0      0.0   1800.0     1800.0
      600864   2500.0      0.0   2500.0     2500.0
      600895   8000.0   1000.0   7000.0     8000.0
      601038  13000.0   2600.0  10400.0    13000.0
      601200   1400.0   1400.0      0.0     1400.0
      601211   3300.0      0.0   3300.0     3300.0
      601777  44300.0   5600.0  38700.0    44300.0
      601857  11200.0      0.0  11200.0    11200.0
      601888    900.0    200.0    700.0      900.0
      601919   3700.0      0.0   3700.0     3700.0
      603106   3600.0      0.0   3600.0     3600.0
      603128   2300.0      0.0   2300.0     2300.0
      603369   4600.0    600.0   4000.0     4600.0
      603508    400.0      0.0    400.0      400.0
      603648   8400.0      0.0   8400.0     8400.0
      603686   5000.0   1000.0   4000.0     5000.0
      603777   1400.0      0.0   1400.0     1400.0
      603797   6000.0   1500.0   4500.0     6000.0
      603882    200.0    200.0      0.0      200.0

[9558 rows x 4 columns]

In [25]:
pd.set_option('display.max_rows',None)

In [26]:
(result['real_hold'] - result['ty_hold'])\
[(result['real_hold'] - result['ty_hold']<-100)|(result['real_hold'] - result['ty_hold']>100)].sort_values()[\
(result['real_hold'] - result['ty_hold'])\
[(result['real_hold'] - result['ty_hold']<-100)|(result['real_hold'] - result['ty_hold']>100)].sort_values()<0]\
.reset_index()['产品标识'].value_counts()

HQYH23    15
HQDB3      3
HQGX1      2
HQYH21     2
HQDF1      2
HQHF9      2
HQYH4      2
HQHF6      2
HQHF1      2
HQCC4      2
HQYH28     2
HQHF12     2
HQGD1      2
HQDT1      2
HQYH18     2
HQYH15     2
HQCC3      2
HQZJ3      2
HQDB2      2
HQHT3      2
HQHT7      2
HQHF11     2
HQYH9      2
HQHZ18     2
HQHT14     2
HQXB1      1
HQXB2      1
HQHF10     1
HQHZ7      1
HQHZ21     1
HQCC5      1
HQYH6      1
HQYH7      1
HQYH25     1
HQHZ20     1
HQHT12     1
HQYH2      1
Name: 产品标识, dtype: int64

In [27]:
(result['real_hold'] - result['ty_hold'])\
[(result['real_hold'] - result['ty_hold']<-100)|(result['real_hold'] - result['ty_hold']>100)].sort_values()[\
(result['real_hold'] - result['ty_hold'])\
[(result['real_hold'] - result['ty_hold']<-100)|(result['real_hold'] - result['ty_hold']>100)].sort_values()>0]\
.reset_index()['产品标识'].value_counts()

HQHT3     18
HQHX1     11
HQHT6     10
HQYH18     7
HQYH25     6
HQHT12     5
HQYH11     5
HQYH2      5
HQYH7      4
HQCC3      4
HQHZ18     3
HQDT1      3
HQCC4      3
HQYH15     2
HQHZ7      2
HQZJ3      2
HQHZ20     1
HQYH23     1
Name: 产品标识, dtype: int64

In [28]:
(result['real_hold'] - result['ty_hold'])\
[(result['real_hold'] - result['ty_hold']<-100)|(result['real_hold'] - result['ty_hold']>100)].sort_values()[\
(result['real_hold'] - result['ty_hold'])\
[(result['real_hold'] - result['ty_hold']<-100)|(result['real_hold'] - result['ty_hold']>100)].sort_values()>0]\
.reset_index().groupby('产品标识').sum()[0].sort_values(ascending = False)

产品标识
HQHT3     116600.0
HQHX1      46100.0
HQYH2      44400.0
HQHT6      34300.0
HQHZ7      25900.0
HQYH25     18798.0
HQYH18     14600.0
HQYH7       6700.0
HQYH23      4659.0
HQCC3       3800.0
HQCC4       2600.0
HQDT1       2000.0
HQYH11      2000.0
HQHZ18      1500.0
HQHT12      1100.0
HQHZ20       972.0
HQYH15       900.0
HQZJ3        400.0
Name: 0, dtype: float64

In [29]:
#真实持仓和理论持仓的分账户股数差距
(result['real_hold'] - result['ty_hold'])\
[(result['real_hold'] - result['ty_hold']<-100)|(result['real_hold'] - result['ty_hold']>100)].sort_values()

产品标识    股票代码  
HQYH23  300431   -69900.0
HQYH25  300431   -69600.0
HQHZ7   300362   -48700.0
HQHF10  300362   -44100.0
HQCC5   300362   -42200.0
HQHF9   300362   -41300.0
HQHZ21  300431   -36540.0
HQHT3   300362   -31400.0
HQHF9   300431   -26300.0
HQHT12  300362   -24400.0
HQYH28  300362   -22600.0
HQHT3   300431   -20100.0
HQHT7   300362   -17700.0
HQXB2   300362   -16600.0
HQYH23  2256     -15600.0
HQYH18  300362   -14500.0
HQYH28  300431   -14400.0
HQDB2   300362   -12700.0
HQGX1   300362   -12700.0
HQGD1   300362   -11500.0
HQHT7   300431   -11300.0
HQCC3   300362   -10900.0
HQYH4   300362   -10100.0
HQHT14  300362    -9600.0
HQYH2   300431    -9600.0
HQYH23  600157    -9300.0
HQYH18  300431    -9200.0
HQHF12  300431    -8400.0
HQGX1   300431    -8100.0
HQDB2   300431    -8100.0
HQCC4   300362    -7800.0
HQHZ18  300362    -7700.0
HQDT1   300362    -7700.0
HQGD1   300431    -7300.0
HQYH6   300431    -6900.0
HQHF11  300362    -6800.0
HQYH23  300278    -6666.0
HQYH9   300362    -6400

a = (result['real_hold'] - result['ty_hold'])\
[(result['real_hold'] - result['ty_hold']<-100)|(result['real_hold'] - result['ty_hold']>100)].sort_values().reset_index()[\
(result['real_hold'] - result['ty_hold'])\
[(result['real_hold'] - result['ty_hold']<-100)|(result['real_hold'] - result['ty_hold']>100)].sort_values().reset_index()['产品标识'].isin(['HQDB2','HQYH9'])                                                                                                                           
]
a['证券名称'] = code_to_name.loc[a['股票代码']].tolist()
a.to_excel('dsd.xlsx',index=False)

In [30]:
pd.set_option('display.max_rows',60)

In [31]:
#真实持仓和理论持仓的总股数差距
np.abs((result['real_hold'] - result['ty_hold'])\
[(result['real_hold'] - result['ty_hold']<-100)|(result['real_hold'] - result['ty_hold']>100)].sort_values()).sum()

1167474.0

In [32]:
'''
制作卖出的逻辑！！！
以可T为校准目标，即，max(真实持仓 - 理论可T, 0) 作为明日的卖出部分，余下的为明日可T部分。
此方法下明日可T会基本与理论可T保持一致，除非股数不够。
****不会进行补足****
****不建议补足，除非对卖出日期可以有明确的记录****
'''
print()

In [33]:
#上述的正常处理
sell = result['real_hold'] - result['ty_t0']
sell = sell[sell>0]
sell = sell.loc[result.index].fillna(0)
t0 = result['real_hold'] - sell

In [34]:
sell = sell[sell>99]
t0 = t0[t0>0]
sell = sell//100*100

In [35]:
t0 = t0.reset_index()
sell = sell.reset_index()

In [36]:
os.chdir(r'C:/Users/bj/Desktop/刘泽丹/Code/')

In [37]:
#输出可T文件，为Code文件夹下名为‘DATE可T券单’的文件夹
t0.columns = ['产品标识','股票代码','任务股数']
os.chdir(r'C:/Users/bj/Desktop/刘泽丹/Code/')
try:
    os.chdir(tomorrow_nodot+'可T券单')
except FileNotFoundError:
    os.mkdir(tomorrow_nodot+'可T券单')
    os.chdir(tomorrow_nodot+'可T券单')
for code in t0['产品标识'].unique():
    if 'A52' not in account_name_to_t0code.xs(account_code_to_name.xs(code)):
        sub_t0 = t0[t0['产品标识'] == code]\
            .loc[:,['股票代码','股票名称','任务股数']].sort_values('任务股数',ascending=False)
        sub_t0.columns = ['股票代码','股票名称','可T+0股数']
        sub_t0['股票名称'] = code_to_name.loc[sub_t0['股票代码']].tolist()
        try:
            sub_t0.to_excel(tomorrow_nodot+'_'+account_name_to_t0code.xs(account_code_to_name.xs(code))+\
                          ' 可T股数查询.xlsx',encoding='gbk',index=False)
            print(account_code_to_name.xs(code),code,sub_t0['可T+0股数'].sum())
        except:
            print('******************',account_code_to_name.xs(code))
    else:
        sub_t0 = t0[t0['产品标识'] == code]\
            .loc[:,['股票代码','股票名称','任务股数']].sort_values('任务股数',ascending=False)
        sub_t0.columns = ['股票代码','股票名称','可T+0股数']
        sub_t0['股票名称'] = code_to_name.loc[sub_t0['股票代码']].tolist()
        sub_t0['股票代码'] = ['0'*(6-len(str(i)))+str(i) for i in sub_t0['股票代码']]
        sub_t0[''] = ''
        sub_t0['资源类型'] = '股票'
        sub_t0['券商账户标识'] = account_name_to_t0code.xs(account_code_to_name.xs(code))
        sub_t0 = sub_t0[['券商账户标识','股票代码','可T+0股数','','资源类型']]
        sub_t0.columns = ['券商账户标识','股票代码','总券数','','资源类型']
        try:
            sub_t0.to_excel(tomorrow_nodot+'_'+account_name_to_t0code.xs(account_code_to_name.xs(code))+\
                          ' 可T股数查询.xlsx',encoding='gbk',index=False)
            print(account_code_to_name.xs(code),code,sub_t0['总券数'].sum())
        except:
            print('******************',account_code_to_name.xs(code))
os.chdir(r'C:/Users/bj/Desktop/刘泽丹/Code/')

金湖聚乐盈1号 HQCC3 1874620.0
金湖聚乐盈3号 HQCC4 1443100.0
金湖AI对冲五十三期长城 HQCC5 8425765.0
金湖增强二期 HQDB2 2049700.0
红墙泰和指数增强六期 HQDB3 1261000.0
稳健二十五期 HQDF1 953900.0
千广一号 HQDT1 1295484.0
金湖AI对冲五十九期 HQGD1 1153700.0
金湖AI增强指数三期 HQGX1 3038750.0
红墙泰和私享一号 HQGX8 19672900.0
陈德英 HQHF1 582809.0
敖麓谷雅二期 HQHF10 6540700.0
敖麓谷雅三期 HQHF11 1098000.0
少伯二号华福 HQHF12 1403300.0
红墙泰和私享三号 HQHF13 135600.0
金湖AI对冲二十八期 HQHF6 647400.0
金湖AI对冲十八期 HQHF7 881200.0
敖麓谷雅一期 HQHF9 6431111.0
金湖宏观量化一期 HQHT11 799700.0
金湖AI对冲五十八期分仓 HQHT12 3637300.0
无量稳健九号 HQHT14 2243409.0
金湖AI对冲十三期 HQHT3 5175401.0
金湖一号 HQHT6 957400.0
金湖AI对冲二十九期 HQHT7 2715500.0
红墙对冲九号 HQHV1 899100.0
红墙泰和稳健十六期 HQHX1 122700.0
红墙泰和私享四号 HQHZ15 101100.0
红墙泰和私享五号 HQHZ16 137000.0
红墙泰和私享六号 HQHZ17 141000.0
黎悦一号 HQHZ18 1148158.0
金湖AI对冲三十六期华泰 HQHZ20 899740.0
元涞阿尔法一号 HQHZ21 2573000.0
金湖AI对冲五十一期 HQHZ7 6473303.0
圭源金湖启航 HQHZ8 78500.0
红墙泰和山丹丹私募 HQXB1 382300.0
无量私享增强指数西部 HQXB2 2707900.0
金湖AI对冲五十期 HQYH11 3278000.0
金湖AI对冲三十七期 HQYH15 902100.0
私享六号 HQYH18 3247300.0
私享二号 HQYH19 5575400.0
金湖AI对冲二十二期 H

In [38]:
#输出卖出文件，为Code文件夹下名为‘总卖出_DATE.xlsx’的文件
os.chdir(r'C:/Users/bj/Desktop/刘泽丹/Code/')
sell.columns = ['产品标识','股票代码','任务股数']
sell['任务资金'] = np.nan
sell['方向'] = '空'
sell = sell.loc[:,['产品标识','股票代码','任务资金','任务股数','方向']]
sell = sell[sell['任务股数']!=0]
sell.to_excel('总任务管理表_卖出.xlsx',encoding='gbk',index=False)

In [39]:
os.chdir(r'C:/Users/bj/Desktop/刘泽丹/Code/')

In [40]:
#检查卖出、可T文件中是否包含记录中应已清仓的

In [41]:
cleared = ['HQAX1','HQAX3','HQCC2','HQCT1','HQCT2','HQCT4','HQDB1','HQGD2','HQGJ1','HQGJ2','HQGS1',\
           'HQGS2','HQGX5','HQGX6','HQGX7','HQHF4','HQHF8','HQHT17','HQHT19','HQHT8','HQHY1',\
           'HQHZ11','HQHZ13','HQHZ14','HQHZ19','HQHZ2','HQHZ3','HQHZ4','HQHZ6','HQHZ9','HQLX1','HQPA1','HQPA2','HQYH1',\
           'HQYH14','HQYH16','HQYH17','HQYH20','HQYH22','HQYH27','HQYH5','HQZT1','HQZX3',\
           'HQHT4','HQCT5','HQXXX1']

In [42]:
t0[t0['产品标识'].isin(cleared)]

,产品标识,股票代码,任务股数


In [43]:
sell[sell['产品标识'].isin(cleared)]

,产品标识,股票代码,任务资金,任务股数,方向


In [44]:
pd.read_excel('买入_未完成_'+today_nodot+'.xlsx')\
[pd.read_excel('买入_未完成_'+today_nodot+'.xlsx')['产品标识'].isin(cleared)]

,产品标识,股票代码,任务资金,任务股数,方向


In [45]:
pd.read_excel('实验_总任务管理表_买入_未完成_'+today_nodot+'.xlsx')\
[pd.read_excel('实验_总任务管理表_买入_未完成_'+today_nodot+'.xlsx')['产品标识'].isin(cleared)]

,产品标识,股票代码,模型,任务资金,完成资金


In [46]:
#卖出中的ST
((result//100*100).reset_index()[result.reset_index()['股票代码'].isin(STcode)].groupby('股票代码').sum()['real_hold']).replace(0,np.nan).dropna()

股票代码
300362     12100.0
300431    518700.0
Name: real_hold, dtype: float64